In [5]:
# FINAL ROBUST GUI (direct final-file writes, per-file + total progress, uses session if present)
# Paste into one notebook cell in your tg_gui env and run.
import nest_asyncio
nest_asyncio.apply()

import asyncio, json, os, time, traceback
from pathlib import Path
from dataclasses import dataclass
from typing import List, Optional
from telethon import TelegramClient, functions, errors, types
from telethon.errors import SessionPasswordNeededError
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------------------- CONFIG --------------------
API_ID = ENTER_YOUR_API_ID_HERE
API_HASH = "ENTER YOUR HASH KEY"
INVITE_LINK = "ENTER CHANNEL NAME/ INVITE LINK/ USERNAME"
SESSION = "tg_gui_env_session"                 # session file prefix (stored, reused)
OUTDIR = Path.home() / "Desktop" / "telegram_downloads"
CHECKPOINT = Path("downloaded_gui_checkpoint.json")
OUTDIR.mkdir(parents=True, exist_ok=True)
# sequential downloads (one at a time)
# ------------------------------------------------

@dataclass
class MediaEntry:
    msg: types.Message
    name: str
    size: Optional[int]

# helpers
def load_checkpoint() -> dict:
    if CHECKPOINT.exists():
        try:
            return json.loads(CHECKPOINT.read_text(encoding="utf8"))
        except Exception:
            return {}
    return {}

def save_checkpoint(data: dict):
    CHECKPOINT.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf8")

def sanitize_filename(name: str) -> str:
    s = name.replace("/", "_").replace("\\", "_")
    s = s.replace(":", "_").replace("?", "_").replace("*", "_").replace('"', "_").replace("<", "_").replace(">", "_").replace("|", "_")
    s = s.strip()
    return s or "unnamed"

def make_name(msg: types.Message) -> str:
    f = getattr(msg, "file", None)
    if f and getattr(f, "name", None):
        return sanitize_filename(f.name)
    stamp = msg.date.strftime("%Y%m%d_%H%M%S") if getattr(msg, "date", None) else f"id{msg.id}"
    ext = ".mp4" if getattr(msg, "video", None) else ""
    return sanitize_filename(f"msg_{msg.id}_{stamp}{ext}")

def is_pdf_or_video(msg: types.Message) -> bool:
    if not msg:
        return False
    if getattr(msg, "video", None):
        return True
    if getattr(msg, "document", None):
        mime = getattr(msg.document, "mime_type", "") or ""
        fname = getattr(msg.file, "name", "") or ""
        if mime.startswith("video") or mime == "application/pdf" or fname.lower().endswith(".pdf") or fname.lower().endswith(".mp4"):
            return True
    return False

# Telethon client global
_client: Optional[TelegramClient] = None

async def get_client():
    """Return connected client; do not prompt in background. Use session if available."""
    global _client
    if _client is None:
        _client = TelegramClient(SESSION, API_ID, API_HASH)
        await _client.connect()
    # if authorized return
    try:
        if await _client.is_user_authorized():
            return _client
    except Exception:
        pass
    return _client

# ---------- UI widgets ----------
out_logs = widgets.Output(layout={'border':'1px solid gray','height':'280px','overflow':'auto'})
display_header = widgets.HTML("<h3>Telegram Downloader — Desktop/telegram_downloads (direct file writes)</h3>")
display(display_header)

# Login widgets (visible only if not authorized)
phone = widgets.Text(placeholder="+91XXXXXXXXXX", description="Phone:", layout=widgets.Layout(width='40%'))
send_code = widgets.Button(description="Send code", button_style='primary')
code = widgets.Text(placeholder="12345", description="Code:", layout=widgets.Layout(width='30%'))
verify = widgets.Button(description="Verify", button_style='success')
logout_btn = widgets.Button(description="Logout", button_style='danger')
login_box = widgets.HBox([phone, send_code, code, verify, logout_btn])
status_label = widgets.HTML("<i>Not logged in</i>")

# Controls
refresh_btn = widgets.Button(description="Refresh list", button_style='info')
select_all_btn = widgets.Button(description="Select all", button_style='warning')
deselect_all_btn = widgets.Button(description="Deselect all")
start_btn = widgets.Button(description="Start download", button_style='success')
cancel_btn = widgets.Button(description="Cancel", button_style='danger')
files_select = widgets.SelectMultiple(options=[], rows=18, layout=widgets.Layout(width='96%'))

controls = widgets.HBox([refresh_btn, select_all_btn, deselect_all_btn, start_btn, cancel_btn])
display(login_box)
display(status_label)
display(controls)
display(files_select)
display(widgets.Label("Logs & detailed status:"))
display(out_logs)

# per-file UI container (dynamic, shows progress bars)
progress_container = widgets.VBox([])
display(progress_container)

# internal state
_media_entries: List[MediaEntry] = []
_cancel_requested = False

# ---------- login flow (widget-driven) ----------
async def send_code_click(_):
    with out_logs:
        print("Sending code to", phone.value)
    try:
        client = await get_client()
        await client.send_code_request(phone.value.strip())
        with out_logs:
            print("Code sent. Paste in 'Code' field and click Verify.")
    except Exception as e:
        with out_logs:
            print("send_code error:", type(e).__name__, e)
            traceback.print_exc()

async def verify_click(_):
    global _client
    with out_logs:
        print("Verifying...")
    try:
        client = await get_client()
        # sign in using code
        await client.sign_in(phone=phone.value.strip(), code=code.value.strip())
    except SessionPasswordNeededError:
        # ask inline (small prompt)
        pwd = widgets.Password(description="2FA pwd")
        btn = widgets.Button(description="Submit 2FA")
        box = widgets.VBox([widgets.HTML("<b>Two-step password required</b>"), pwd, btn])
        display(box)
        def on_submit(b):
            async def _do():
                try:
                    await client.sign_in(password=pwd.value)
                    box.close()
                    await post_login_setup()
                except Exception as e:
                    with out_logs:
                        print("2FA failed:", e)
            asyncio.create_task(_do())
        btn.on_click(on_submit)
        return
    except Exception as e:
        with out_logs:
            print("verify error:", type(e).__name__, e)
            traceback.print_exc()
        return
    # successful?
    if await client.is_user_authorized():
        await post_login_setup()

async def post_login_setup():
    client = await get_client()
    if await client.is_user_authorized():
        status_label.value = "<b style='color:green'>Logged in</b>"
        with out_logs:
            print("Logged in and session saved; you won't need to login again unless you logout.")
        # hide login fields (optional)
        phone.layout.display = 'none'
        send_code.layout.display = 'none'
        code.layout.display = 'none'
        verify.layout.display = 'none'

def logout_click(_):
    # remove session files and restart client
    global _client, _media_entries
    try:
        # close client
        if _client:
            asyncio.create_task(_client.disconnect())
            _client = None
    except Exception:
        pass
    # delete session files
    for p in Path('.').glob(f"{SESSION}*"):
        try:
            p.unlink()
        except Exception:
            pass
    status_label.value = "<i>Not logged in</i>"
    phone.layout.display = None
    send_code.layout.display = None
    code.layout.display = None
    verify.layout.display = None
    with out_logs:
        print("Logged out and session files removed.")
    # clear list
    files_select.options = []
    _media_entries = []

# wire buttons
send_code.on_click(lambda b: asyncio.create_task(send_code_click(b)))
verify.on_click(lambda b: asyncio.create_task(verify_click(b)))
logout_btn.on_click(logout_click)

# disable refresh until authorized (but if session exists it will be enabled)
async def maybe_enable_refresh():
    client = await get_client()
    try:
        if await client.is_user_authorized():
            status_label.value = "<b style='color:green'>Logged in (session available)</b>"
            phone.layout.display = 'none'
            send_code.layout.display = 'none'
            code.layout.display = 'none'
            verify.layout.display = 'none'
            return True
    except Exception:
        pass
    return False

asyncio.create_task(maybe_enable_refresh())

# ---------- scanning channel ----------
def invite_hash(link: str) -> str:
    return link.rstrip("/").split("/")[-1].lstrip("+")

async def refresh_list_click(_):
    global _media_entries
    with out_logs:
        clear_output()
        print("Refreshing list... (this may take a while)")
    try:
        client = await get_client()
        if not await client.is_user_authorized():
            with out_logs:
                print("Client not authorized. Use login UI first.")
            return
        # join or resolve
        try:
            ent = await client(functions.messages.ImportChatInviteRequest(hash=invite_hash(INVITE_LINK)))
            entity = ent.chats[0] if getattr(ent, "chats", None) else await client.get_entity(INVITE_LINK)
        except Exception:
            entity = await client.get_entity(INVITE_LINK)
        with out_logs:
            print("Channel resolved:", getattr(entity, "title", getattr(entity, "username", str(entity))))
            print("Scanning for PDFs and videos ...")
        entries = []
        async for msg in client.iter_messages(entity, reverse=True):
            if is_pdf_or_video(msg):
                entries.append(MediaEntry(msg=msg, name=make_name(msg), size=getattr(msg.file, "size", None) if getattr(msg, "file", None) else None))
        _media_entries = entries
        # prepare options labels so user sees size
        files_select.options = [(f"{i+1:03d}. {e.name} ({e.size or 'unknown'} bytes)", i) for i,e in enumerate(entries)]
        with out_logs:
            print("Scan complete. Found", len(entries), "items.")
    except Exception as e:
        with out_logs:
            print("Refresh error:", type(e).__name__, e)
            traceback.print_exc()

refresh_btn.on_click(lambda b: asyncio.create_task(refresh_list_click(b)))

# select helpers
select_all_btn.on_click(lambda b: setattr(files_select, 'value', tuple([opt[1] for opt in files_select.options])))
deselect_all_btn.on_click(lambda b: setattr(files_select, 'value', ()))

# ---------- download single-file with direct final-file write (no .part) ----------
def human(n):
    # human-readable bytes
    for u in ['B','KB','MB','GB','TB']:
        if n < 1024.0:
            return f"{n:3.1f}{u}"
        n /= 1024.0
    return f"{n:.1f}PB"

async def download_direct(client: TelegramClient, entry: MediaEntry, target_path: Path, ui_bar: widgets.FloatProgress, ui_label: widgets.Label, overall_cb):
    """Download directly into target_path (no .part). Uses Telethon progress_callback to update widgets."""
    # skip if already exists with same size from checkpoint
    cp = load_checkpoint()
    rec = cp.get(str(entry.msg.id))
    if rec and target_path.exists():
        try:
            if rec.get("size") is None or target_path.stat().st_size == rec.get("size"):
                ui_label.value = f"Skipped (exists) — {human(target_path.stat().st_size)}"
                return ("skipped", str(target_path))
        except Exception:
            pass

    # perform download directly to target path
    start_time = time.time()
    last_received = 0
    last_time = start_time

    def progress_cb(received, total):
        nonlocal last_received, last_time
        # update UI widgets from the callback (in same thread)
        try:
            # percentage
            pct = (received / total * 100) if total else 0.0
            ui_bar.value = pct
            # compute instantaneous speed
            now = time.time()
            delta_b = received - last_received
            delta_t = now - last_time
            if delta_t <= 0:
                speed = 0.0
            else:
                speed = delta_b / max(delta_t, 1e-6)
            last_received = received
            last_time = now
            # update label
            ui_label.value = f"{human(received)} / {human(total or 0)} — {pct:5.1f}% @ {human(speed)}/s"
            # update overall progress
            if overall_cb:
                overall_cb(delta_b)
        except Exception:
            pass

    try:
        # IMPORTANT: this writes directly to 'target_path' (no .part)
        await client.download_media(entry.msg, file=str(target_path), progress_callback=progress_cb)
        # final check
        final_size = target_path.stat().st_size if target_path.exists() else 0
        ui_label.value = f"Done — {human(final_size)}"
        # update checkpoint
        cp = load_checkpoint()
        cp[str(entry.msg.id)] = {"name": str(target_path.name), "size": final_size}
        save_checkpoint(cp)
        return ("downloaded", str(target_path))
    except errors.FloodWaitError as fe:
        with out_logs:
            print("FloodWait (need to wait):", fe)
        return ("failed", "floodwait")
    except Exception as e:
        with out_logs:
            print("Download direct error:", type(e).__name__, e)
            traceback.print_exc()
        return ("failed", str(e))

# ---------- overall download flow ----------
async def download_flow():
    global _cancel_requested
    sel = files_select.value
    if not sel:
        with out_logs:
            print("No files selected.")
        return
    indices = [int(i) for i in sel]
    selected = [_media_entries[i] for i in indices if 0<=i<len(_media_entries)]
    if not selected:
        with out_logs:
            print("Selection invalid.")
        return

    client = await get_client()
    if not await client.is_user_authorized():
        with out_logs:
            print("Client not authorized; use login UI or verify.")
        return

    # prepare UI progress widgets per selected file
    progress_widgets = []
    for e in selected:
        bar = widgets.FloatProgress(value=0.0, min=0.0, max=100.0, description=e.name[:40], layout=widgets.Layout(width='90%'))
        label = widgets.Label(value="Queued")
        row = widgets.HBox([bar, label])
        progress_widgets.append((e, bar, label, row))
    # show them
    progress_container.children = [r for (_,_,_,r) in progress_widgets]

    # overall bar
    total_known = sum(e.size for e in selected if e.size)
    overall_bar = widgets.FloatProgress(value=0.0, min=0.0, max=100.0, description="TOTAL", layout=widgets.Layout(width='96%'))
    overall_label = widgets.Label(value="0 / 0")
    progress_container.children = list(progress_container.children) + [widgets.HBox([overall_bar, overall_label])]

    # helper to update overall
    overall_received = 0
    def overall_cb(delta):
        nonlocal overall_received
        overall_received += delta
        if total_known:
            overall_pct = min(100.0, overall_received / total_known * 100.0)
            overall_bar.value = overall_pct
            overall_label.value = f"{human(overall_received)} / {human(total_known)} ({overall_pct:4.1f}%)"
        else:
            overall_label.value = f"{human(overall_received)} transferred"

    # run downloads sequentially
    _cancel_requested = False
    succ = skipped = fail = 0
    for (entry, bar, label, _) in progress_widgets:
        if _cancel_requested:
            with out_logs:
                print("Cancellation requested — stopping after current file.")
            break
        label.value = "Starting..."
        target = OUTDIR / entry.name
        # ensure path sanitized / shorten
        try:
            target = Path(str(target))  # ensure Path
        except Exception:
            target = OUTDIR / ("file_" + str(entry.msg.id))
        res = await download_direct(client, entry, target, bar, label, overall_cb)
        if res[0] == "downloaded":
            succ += 1
        elif res[0] == "skipped":
            skipped += 1
        else:
            fail += 1
    with out_logs:
        print(f"Download completed. Success: {succ} | Skipped: {skipped} | Failed: {fail}")
        print("Files on Desktop:", str(OUTDIR.resolve()))
    # reset UI state
    progress_container.children = []

# wire controls
def on_start(_):
    start_btn.disabled = True
    cancel_btn.disabled = False
    refresh_btn.disabled = True
    asyncio.create_task(download_flow()).add_done_callback(lambda fut: (
        setattr(start_btn, 'disabled', False),
        setattr(cancel_btn, 'disabled', True),
        setattr(refresh_btn, 'disabled', False)
    ))

def on_cancel(_):
    global _cancel_requested
    _cancel_requested = True
    with out_logs:
        print("Cancel requested; will stop after current file.")

start_btn.on_click(on_start)
cancel_btn.on_click(on_cancel)

def on_logout(_):
    logout_click(None)

logout_btn.on_click(on_logout)

# initial state: enable refresh if session present
async def init_state():
    try:
        client = await get_client()
        if await client.is_user_authorized():
            status_label.value = "<b style='color:green'>Logged in</b>"
            phone.layout.display = 'none'
            send_code.layout.display = 'none'
            code.layout.display = 'none'
            verify.layout.display = 'none'
    except Exception:
        pass

asyncio.create_task(init_state())

# final message
with out_logs:
    print("UI ready. If you already logged in earlier the session will be reused.")
    print("Use Login -> Send code -> Verify only the first time. Then Refresh, select files and Start.")
    print("Files will be saved directly to Desktop/telegram_downloads (no .part files).")


HTML(value='<h3>Telegram Downloader — Desktop/telegram_downloads (direct file writes)</h3>')

HTML(value='<i>Not logged in</i>')

SelectMultiple(layout=Layout(width='96%'), options=(), rows=18, value=())

Label(value='Logs & detailed status:')

Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

VBox()